---
# Relatório automatizado dos principais resultados da semana
---

**Instalação e importação de bibliotecas**

---

In [ ]:
pip install yfinance

In [ ]:
pip install kaleido

In [3]:
import numpy as np
import pandas as pd
from datetime import datetime, timedelta
import pytz
import yfinance as yf
import kaleido
import plotly.graph_objects as go
from plotly.subplots import make_subplots

**Organizar datas para extração**

---
*   Precisamos obter as datas de forma automatizada, para serem utilizadas na extração dos dados.

In [4]:
hoje = datetime.now()   # informação completa sobre data e hora atual
hoje.weekday()          # coleta o dia da semana, iniciando na segunda feira (dia 0)

4

In [5]:
# padronizando o dia inicial como dia zero
segundafeira = hoje - timedelta(days = hoje.weekday())
# selecionando somente a data
segundafeira.date()

datetime.date(2024, 6, 17)

In [6]:
# transformando data em string compatível para uso na yfinance
data_inicial = segundafeira.date().strftime('%Y-%m-%d')
# variável para data final, transformando para string. Usamos +2 para garantir que todos os dados serão coletados na sexta-feira
data_final = (hoje.date() + timedelta(days = 2)).strftime('%Y-%m-%d')
data_final

'2024-06-23'

In [7]:
# função para geração da capa/cabeçalho da postagem do relatório
def capa_postagem_semanal():
    # coletando a semana do ano, em string
    semana_ano = segundafeira.date().strftime('%U')
    # coletando o ano, em string
    ano = segundafeira.date().strftime('%Y')
    # emissão das informações
    print('Guilherme Nogueira apresenta')
    print('Weekly Report')
    print(f'Semana {semana_ano} de {ano}')
    print(f'{segundafeira.date().strftime("%d-%m")} a {(segundafeira.date() + timedelta(days = 4)).strftime("%d-%m")}')

**Índices internacionais**

---
*   Extração e manipulação de dados

In [42]:
# dicionário com as informações desejadas (chaves e valores em lista)
dic_indices_internacionais = {
    'Bovespa (BVSP)': ['^BVSP','Brasil'],
    'S&P 500 (GSPC)': ['^GSPC', 'EUA'],
    'Dow Jones 30 (DJI)': ['^DJI', 'EUA'],
    'NASDAQ Composite (IXIC)': ['^IXIC', 'EUA'],
    'S&P/TSX Composite (GSPTSE)': ['^GSPTSE', 'Canadá'],
    'S&P/BMV IPC (MXX)': ['^MXX', 'México'],
    'S&P Merval (MERV)': ['^MERV', 'Argentina'],
    'Shanghai Composite (SSEC)': ['000001.SS', 'China'],
    'SZSE Component (SZI)': ['399001.SZ', 'China'],
    'Nikkei 225 (N225)': ['^N225', 'Japão'],
    'Hang Seng (HK50)': ['^HSI', 'Hong Kong'],
    'BSE Sensex 30 (BSESN)': ['^BSESN', 'Índia'],
    'EuroNext 100 (N100)': ['^N100', 'Zona Euro'],
    'DAX (DE30)': ['^GDAXI', 'Alemanha'],
    'FTSE 100 (UK100)': ['^FTSE', 'UK'],
    'CAC 40 (FCHI)': ['^FCHI', 'França'],
    }

dic_indices_internacionais

{'Bovespa (BVSP)': ['^BVSP', 'Brasil'],
 'S&P 500 (GSPC)': ['^GSPC', 'EUA'],
 'Dow Jones 30 (DJI)': ['^DJI', 'EUA'],
 'NASDAQ Composite (IXIC)': ['^IXIC', 'EUA'],
 'S&P/TSX Composite (GSPTSE)': ['^GSPTSE', 'Canadá'],
 'S&P/BMV IPC (MXX)': ['^MXX', 'México'],
 'S&P Merval (MERV)': ['^MERV', 'Argentina'],
 'Shanghai Composite (SSEC)': ['000001.SS', 'China'],
 'SZSE Component (SZI)': ['399001.SZ', 'China'],
 'Nikkei 225 (N225)': ['^N225', 'Japão'],
 'Hang Seng (HK50)': ['^HSI', 'Hong Kong'],
 'BSE Sensex 30 (BSESN)': ['^BSESN', 'Índia'],
 'EuroNext 100 (N100)': ['^N100', 'Zona Euro'],
 'DAX (DE30)': ['^GDAXI', 'Alemanha'],
 'FTSE 100 (UK100)': ['^FTSE', 'UK'],
 'CAC 40 (FCHI)': ['^FCHI', 'França']}

In [43]:
# transformando dicionário em dataframe
df_indices_internacionais = pd.DataFrame.from_dict(dic_indices_internacionais, orient='index', columns=['Ticker_YF', 'País'])
df_indices_internacionais.head()

,Ticker_YF,País
Bovespa (BVSP),^BVSP,Brasil
S&P 500 (GSPC),^GSPC,EUA
Dow Jones 30 (DJI),^DJI,EUA
NASDAQ Composite (IXIC),^IXIC,EUA
S&P/TSX Composite (GSPTSE),^GSPTSE,Canadá


In [44]:
# criando novo índice para o dataframe
df_indices_internacionais = df_indices_internacionais.reset_index()
df_indices_internacionais = df_indices_internacionais.rename({'index':'Índice'}, axis=1)
df_indices_internacionais.head()

,Índice,Ticker_YF,País
0,Bovespa (BVSP),^BVSP,Brasil
1,S&P 500 (GSPC),^GSPC,EUA
2,Dow Jones 30 (DJI),^DJI,EUA
3,NASDAQ Composite (IXIC),^IXIC,EUA
4,S&P/TSX Composite (GSPTSE),^GSPTSE,Canadá


In [45]:
# coleta de informações diárias de preço dos índices do dataframe (teste usando somente o primeiro índice do dataframe)
ohlcv_indices = pd.DataFrame(yf.download(df_indices_internacionais.Ticker_YF.values[0], start=data_inicial, end=data_final, progress=False, interval="1d"))
ohlcv_indices

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2024-06-17,119663.000000,119663.000000,118685.000000,119138.000000,119138.000000,7380600
2024-06-18,119138.000000,120109.000000,118872.000000,119630.000000,119630.000000,8100100
2024-06-19,119630.000000,120383.000000,118960.000000,120261.000000,120261.000000,5646300
2024-06-20,120267.000000,121607.000000,120156.000000,120446.000000,120446.000000,8659500
2024-06-21,120446.023438,121580.046875,120061.039062,121341.132812,121341.132812,0


In [46]:
# fazendo redimensionamento (de diário para semanal) e renomeando colunas
ohlcv_indices = ohlcv_indices.resample("W").agg({"Open": "first", "High": "max", "Low": "min", "Close": "last"})
ohlcv_indices["Ticker_YF"] = df_indices_internacionais.Ticker_YF.values[0]
ohlcv_indices

,Open,High,Low,Close,Ticker_YF
Date,,,,,
2024-06-23,119663.0,121607.0,118685.0,121341.132812,^BVSP


In [47]:
# calculando resultado e criando nova coluna para ele
ohlcv_indices["Resultado_%"] = (ohlcv_indices.Close/ohlcv_indices.Open - 1) * 100
ohlcv_indices

,Open,High,Low,Close,Ticker_YF,Resultado_%
Date,,,,,,
2024-06-23,119663.0,121607.0,118685.0,121341.132812,^BVSP,1.402382


In [48]:
# Agora faremos a coleta para todo o dataframe de índices
full_ohlcv = pd.DataFrame()
# usando loop de repetição, que vai buscando os índices 1 a 1 de acordo com o ticker
for i in df_indices_internacionais.Ticker_YF:

    try:
        ohlcv_indices = pd.DataFrame(yf.download(i, start=data_inicial, end=data_final, progress=False, interval="1d"))
        ohlcv_indices = ohlcv_indices.resample("W").agg({"Open": "first", "High": "max", "Low": "min", "Close": "last"})
        ohlcv_indices["Resultado_%"] = (ohlcv_indices.Close/ohlcv_indices.Open - 1) * 100
        ohlcv_indices["Ticker_YF"] = i
        full_ohlcv = pd.concat([full_ohlcv, ohlcv_indices], axis=0)

    except:
        pass

In [50]:
full_ohlcv

,Open,High,Low,Close,Resultado_%,Ticker_YF
Date,,,,,,
2024-06-23,1.196630e+05,1.216070e+05,1.186850e+05,1.213411e+05,1.402382,^BVSP
2024-06-23,5.431110e+03,5.505530e+03,5.420400e+03,5.464620e+03,0.617006,^GSPC
2024-06-23,3.856518e+04,3.925718e+04,3.843195e+04,3.915033e+04,1.517297,^DJI
2024-06-23,1.769730e+04,1.793679e+04,1.762057e+04,1.768936e+04,-0.044863,^IXIC
2024-06-23,2.158430e+04,2.170810e+04,2.146660e+04,2.155486e+04,-0.136402,^GSPTSE
2024-06-23,5.224733e+04,5.354749e+04,5.200077e+04,5.278850e+04,1.035789,^MXX
2024-06-23,1.582372e+06,1.583671e+06,1.548910e+06,1.576462e+06,-0.373490,^MERV
2024-06-23,3.017717e+03,3.031976e+03,2.985929e+03,2.998138e+03,-0.648813,000001.SS
2024-06-23,9.209460e+03,9.329990e+03,9.008339e+03,9.064845e+03,-1.570290,399001.SZ


In [51]:
# unindo os dataframes seguindo o conteúdo da coluna "Ticker_TF", que é a coluna em comum nos dois dataframes
world_indices = pd.merge(df_indices_internacionais, full_ohlcv, on="Ticker_YF", how="outer")
world_indices.head()

,Índice,Ticker_YF,País,Open,High,Low,Close,Resultado_%
0,Bovespa (BVSP),^BVSP,Brasil,119663.000000,121607.000000,118685.000000,121341.132812,1.402382
1,S&P 500 (GSPC),^GSPC,EUA,5431.109863,5505.529785,5420.399902,5464.620117,0.617006
2,Dow Jones 30 (DJI),^DJI,EUA,38565.179688,39257.179688,38431.949219,39150.328125,1.517297
3,NASDAQ Composite (IXIC),^IXIC,EUA,17697.300781,17936.789062,17620.568359,17689.361328,-0.044863
4,S&P/TSX Composite (GSPTSE),^GSPTSE,Canadá,21584.300781,21708.099609,21466.599609,21554.859375,-0.136402


**Visualização gráfica**

In [52]:
# criando dataframe para a figura, ordenando dados pelo resultado
df_cor = world_indices.sort_values(by="Resultado_%", ascending=False)
# nova coluna para cor, onde negativo será vermelho e positivo será verde
df_cor["Cor"] = np.where(df_cor["Resultado_%"] < 0, "red", "green")
df_cor.head()

,Índice,Ticker_YF,País,Open,High,Low,Close,Resultado_%,Cor
2,Dow Jones 30 (DJI),^DJI,EUA,38565.179688,39257.179688,38431.949219,39150.328125,1.517297,green
12,EuroNext 100 (N100),^N100,Zona Euro,1476.660034,1512.479980,1474.520020,1498.030029,1.447184,green
0,Bovespa (BVSP),^BVSP,Brasil,119663.000000,121607.000000,118685.000000,121341.132812,1.402382,green
15,CAC 40 (FCHI),^FCHI,França,7539.029785,7682.640137,7482.379883,7628.569824,1.187687,green
14,FTSE 100 (UK100),^FTSE,UK,8146.899902,8282.660156,8120.899902,8237.719727,1.114778,green


In [53]:
# construindo área de plotagem
fig_world_indices = go.Figure()
# preenchimento da área com gráfico de barras
fig_world_indices.add_trace(go.Bar(
                        x=df_cor["Resultado_%"],
                        y=df_cor["Índice"],
                        marker_color=df_cor["Cor"],
                        orientation="h",
                        )
                    )

In [54]:
# adicionando detalhes à estrutura gráfica

# listas vazias que vão representar os eixos e suas informações
annotations = []
x_pos = []
y_pos = []
x_neg = []
y_neg = []

# loop de repetição que vai preencher as listas de acordo com o resultado de cada índice
for i in range(len(df_cor)):

    if df_cor["Resultado_%"].iloc[i] > 0:
        y_pos.append(df_cor["Índice"].iloc[i])
        x_pos.append(np.round(df_cor["Resultado_%"].iloc[i], decimals=2))
    else:
        y_neg.append(df_cor["Índice"].iloc[i])
        x_neg.append(np.round(df_cor["Resultado_%"].iloc[i], decimals=2))

In [55]:
# determinando informações dos eixos

# limitando o eixo para que fique adequado ao valor máximo e mínimo
result_max = df_cor["Resultado_%"].max()+1
result_min = df_cor["Resultado_%"].min()-1

# loop de repetição para preencher a lista 'annotations' como dicionário
for yd, xd in zip(y_pos, x_pos):
    annotations.append(
        dict(
            xref="x1",
            yref="y1",
            y=yd,
            x=xd+0.5,
            text=str(xd) + "%",
            font=dict(family="Arial", size=17, color="black"),
            showarrow=False,
        )
    )

for yd, xd in zip(y_neg, x_neg):
    annotations.append(
        dict(
            xref="x1",
            yref="y1",
            y=yd,
            x=xd-0.5,
            text=str(xd) + "%",
            font=dict(family="Arial", size=17, color="black"),
            showarrow=False,
        )
    )

In [56]:
# atualizar eixo X
fig_world_indices.update_xaxes(
        visible=True,
        showticklabels=True,
        range=[(result_min), (result_max)],
    )

# atualizar layout geral da área de plotagem
fig_world_indices = fig_world_indices.update_layout(
        title_text="<b>Resultado semanal",
        template="simple_white",
        margin=dict(l=20, r=20, t=70, b=20),
        paper_bgcolor="#f7f8fa",
        width=800,
        height=(50 * len(df_cor)),
        annotations=annotations,
        font=dict(family="Arial", size=17, color="black"),
    )

In [57]:
fig_world_indices

In [23]:
# exportando imagem com a kaleido
fig_world_indices.write_image("indices_internacionais.png",  width=800, height=1000, scale=5)

In [24]:
# exportando html com a kaleido
fig_world_indices.write_html("indices_internacionais.html")

**Índices nacionais**

---
*   Extração e manipulação de dados

In [58]:
# criação de dicionário com chave e valor
dic_indices_br = {
    "Bovespa": "^BVSP",
    "Índice Futuro": "^IBX50",
    "Índice Brasil": "IBXX.SA",
    "Índice Mid-Large Cap": "MLCX.SA",
    "Índice Small Cap": "SMLL.SA",
    "Índice Brasil Broad-Based": "IBRA.SA",
    "Índice Bovespa Tag Along": "ITAG.SA",
    "Índice Fundos Imobiliários": "IFIX.SA",
    "Índice BDR": "BDRX.SA",
}

In [59]:
# transformando dicionário em dataframe e resetando índice
df_indices_br = pd.DataFrame.from_dict(dic_indices_br, orient='index', columns=['Ticker_YF'])
df_indices_br = df_indices_br.reset_index()
df_indices_br = df_indices_br.rename({'index':'Índice'}, axis=1)
df_indices_br.head()

,Índice,Ticker_YF
0,Bovespa,^BVSP
1,Índice Futuro,^IBX50
2,Índice Brasil,IBXX.SA
3,Índice Mid-Large Cap,MLCX.SA
4,Índice Small Cap,SMLL.SA


In [27]:
# coleta para o dataframe de índices BR
full_ohlcv_br = pd.DataFrame()
# usando loop de repetição, que vai buscando os índices 1 a 1 de acordo com o ticker
for i in df_indices_br.Ticker_YF:

    try:
        ohlcv_indices_br = pd.DataFrame(yf.download(i, period = "5d", progress=False, interval="1d"))
        ohlcv_indices_br = ohlcv_indices_br.resample("W").agg({"Open": "first", "High": "max", "Low": "min", "Close": "last"})
        ohlcv_indices_br["Resultado_%"] = (ohlcv_indices_br.Close/ohlcv_indices_br.Open - 1) * 100
        ohlcv_indices_br["Ticker_YF"] = i
        full_ohlcv_br = pd.concat([full_ohlcv_br, ohlcv_indices_br], axis=0)

    except:
        pass

In [28]:
full_ohlcv_br

,Open,High,Low,Close,Resultado_%,Ticker_YF
Date,,,,,,
2024-06-23,119663.000000,121607.000000,118685.000000,121341.132812,1.402382,^BVSP
2024-06-23,20060.550781,20430.689453,19906.150391,20381.630859,1.600555,^IBX50
2024-06-23,50894.691406,51357.128906,50735.410156,51253.921875,0.705831,IBXX.SA
2024-06-23,2429.750000,2452.510010,2422.600098,2448.040039,0.752754,MLCX.SA
2024-06-23,1943.010010,1960.869995,1933.439941,1954.599976,0.596495,SMLL.SA
2024-06-23,4755.750000,4799.569824,4740.819824,4790.859863,0.738261,IBRA.SA
2024-06-23,26954.269531,27208.330078,26874.539062,27154.400391,0.742483,ITAG.SA
2024-06-23,3301.540039,3309.659912,3298.479980,3299.379883,-0.065429,IFIX.SA
2024-06-23,19314.630859,19317.710938,19022.380859,19140.800781,-0.899992,BDRX.SA


In [29]:
full_ohlcv_br = full_ohlcv_br.set_index('Ticker_YF', drop=True)
full_ohlcv_br.head()

,Open,High,Low,Close,Resultado_%
Ticker_YF,,,,,
^BVSP,119663.000000,121607.000000,118685.000000,121341.132812,1.402382
^IBX50,20060.550781,20430.689453,19906.150391,20381.630859,1.600555
IBXX.SA,50894.691406,51357.128906,50735.410156,51253.921875,0.705831
MLCX.SA,2429.750000,2452.510010,2422.600098,2448.040039,0.752754
SMLL.SA,1943.010010,1960.869995,1933.439941,1954.599976,0.596495


**Dataframe interativo**

---

In [30]:
# fazendo cópia do dataframe, eliminando casas decimais dos preços e arredondando o Resultado %
df = full_ohlcv_br.copy()
df = df.style.format("{:.0f}").format("{:.2f}", subset='Resultado_%')
df

,Open,High,Low,Close,Resultado_%
Ticker_YF,,,,,
^BVSP,119663,121607,118685,121341,1.40
^IBX50,20061,20431,19906,20382,1.60
IBXX.SA,50895,51357,50735,51254,0.71
MLCX.SA,2430,2453,2423,2448,0.75
SMLL.SA,1943,1961,1933,1955,0.60
IBRA.SA,4756,4800,4741,4791,0.74
ITAG.SA,26954,27208,26875,27154,0.74
IFIX.SA,3302,3310,3298,3299,-0.07
BDRX.SA,19315,19318,19022,19141,-0.90


In [31]:
# aplicando esquema de cores de acordo com o resultado usando o applymap
df = full_ohlcv_br.copy()
df = df.style.format("{:.0f}").format("{:.2f}", subset='Resultado_%').applymap(
            lambda x:
                "font-weight: bold; color: white; background-color : #d40202"
                if x < -1
                else (
                    "font-weight: bold; color: white; background-color : #d66363"
                    if x < 0
                    else (
                        "font-weight: bold; color: white; background-color : #4eb55c"
                        if x > 0.42
                        else "font-weight: bold; color: black; background-color : #c1dec5"
                        )
                    ),
            subset=["Resultado_%"],
            )
df

,Open,High,Low,Close,Resultado_%
Ticker_YF,,,,,
^BVSP,119663,121607,118685,121341,1.40
^IBX50,20061,20431,19906,20382,1.60
IBXX.SA,50895,51357,50735,51254,0.71
MLCX.SA,2430,2453,2423,2448,0.75
SMLL.SA,1943,1961,1933,1955,0.60
IBRA.SA,4756,4800,4741,4791,0.74
ITAG.SA,26954,27208,26875,27154,0.74
IFIX.SA,3302,3310,3298,3299,-0.07
BDRX.SA,19315,19318,19022,19141,-0.90


In [32]:
# tornando o dataframe interativo com set_table_styles
df = full_ohlcv_br.copy()
df = df.style.format("{:.0f}").format("{:.2f}", subset='Resultado_%').set_table_styles(
            [
                {"selector": "td", "props": "text-align: center"}, # texto centralizado
                {"selector": "th", "props": "text-align: center; font-weight: bold; background-color: #b8b4b4; color: black; font-size: 14pt"}, # headers
                {"selector": "th.index_name", "props": "text-align: center; font-weight: bold; background-color: red; color: white; font-size: 14pt"}, # nome dos índices
                {"selector": "th.row_heading", "props": "text-align: center; font-weight: bold; background-color: salmon; color: black; font-size: 11pt"}, # cabeçalho das linhas
                {"selector": "th.col_heading", "props": "text-align: center; font-weight: bold; background-color: green; color: black; font-size: 11pt"}, # cabeçalho das colunas
                {"selector": "td:hover", "props": "text-align: center; font-weight: bold; background-color: yellow; color: black; font-size: 15pt"}, # ao passar o mouse, a célula será destacada
            ],
            overwrite=False,
        )

df

,Open,High,Low,Close,Resultado_%
Ticker_YF,,,,,
^BVSP,119663,121607,118685,121341,1.40
^IBX50,20061,20431,19906,20382,1.60
IBXX.SA,50895,51357,50735,51254,0.71
MLCX.SA,2430,2453,2423,2448,0.75
SMLL.SA,1943,1961,1933,1955,0.60
IBRA.SA,4756,4800,4741,4791,0.74
ITAG.SA,26954,27208,26875,27154,0.74
IFIX.SA,3302,3310,3298,3299,-0.07
BDRX.SA,19315,19318,19022,19141,-0.90


In [33]:
# unindo código com as alterações de applymap e set_table_styles
df = full_ohlcv_br.copy()
df = df.style.format("{:.0f}").format("{:.2f}", subset='Resultado_%').applymap(
            lambda x:
                "font-weight: bold; color: white; background-color : #d40202"
                if x < -1
                else (
                    "font-weight: bold; color: white; background-color : #d66363"
                    if x < 0
                    else (
                        "font-weight: bold; color: white; background-color : #4eb55c"
                        if x > 0.42
                        else "font-weight: bold; color: black; background-color : #c1dec5"
                        )
                    ),
            subset=["Resultado_%"],
            ).set_table_styles(
            [
                {"selector": "td", "props": "text-align: center"},
                {"selector": "th", "props": "text-align: center; font-weight: bold; background-color: #b8b4b4; color: black; font-size: 14pt"},
                {"selector": "th.index_name", "props": "text-align: center; font-weight: bold; background-color: red; color: white; font-size: 14pt"},
                {"selector": "th.row_heading", "props": "text-align: center; font-weight: bold; background-color: salmon; color: black; font-size: 11pt"},
                {"selector": "th.col_heading", "props": "text-align: center; font-weight: bold; background-color: green; color: black; font-size: 11pt"},
                {"selector": "td:hover", "props": "text-align: center; font-weight: bold; background-color: yellow; color: black; font-size: 15pt"},
            ],
            overwrite=False,
        )

df

,Open,High,Low,Close,Resultado_%
Ticker_YF,,,,,
^BVSP,119663,121607,118685,121341,1.40
^IBX50,20061,20431,19906,20382,1.60
IBXX.SA,50895,51357,50735,51254,0.71
MLCX.SA,2430,2453,2423,2448,0.75
SMLL.SA,1943,1961,1933,1955,0.60
IBRA.SA,4756,4800,4741,4791,0.74
ITAG.SA,26954,27208,26875,27154,0.74
IFIX.SA,3302,3310,3298,3299,-0.07
BDRX.SA,19315,19318,19022,19141,-0.90


**Candles da semana e últimos 30 dias**

---

In [34]:
hoje = datetime.now()   # informação completa sobre data e hora atual
hoje.weekday()          # coleta o dia da semana, iniciando na segunda feira (dia 0)

4

In [35]:
# padronizando o dia inicial como dia zero
segundafeira = hoje - timedelta(days = hoje.weekday())
# selecionando somente a data
segundafeira.date()

datetime.date(2024, 6, 17)

In [36]:
data_inicial_candles = segundafeira.date().strftime('%Y-%m-%d')
data_final_candles = (hoje.date() + timedelta(days = 2)).strftime('%Y-%m-%d')
inicio_mes = (segundafeira.date() - timedelta(days = 30)).strftime('%Y-%m-%d')
inicio_mes

'2024-05-18'

In [37]:
ohlcv_semana = pd.DataFrame(yf.download("^BVSP", data_inicial_candles, data_final_candles, progress=False, interval="1d"))
ohlcv_semana

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2024-06-17,119663.000000,119663.000000,118685.000000,119138.000000,119138.000000,7380600
2024-06-18,119138.000000,120109.000000,118872.000000,119630.000000,119630.000000,8100100
2024-06-19,119630.000000,120383.000000,118960.000000,120261.000000,120261.000000,5646300
2024-06-20,120267.000000,121607.000000,120156.000000,120446.000000,120446.000000,8659500
2024-06-21,120446.023438,121580.046875,120061.039062,121341.132812,121341.132812,0


In [38]:
ohlcv_mes = pd.DataFrame(yf.download("^BVSP", inicio_mes, data_final_candles, progress=False, interval="1d"))
ohlcv_mes.head()

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2024-05-20,128151.0,128730.0,127488.0,127751.0,127751.0,9340300
2024-05-21,127754.0,128272.0,127205.0,127412.0,127412.0,9138500
2024-05-22,127412.0,127412.0,125524.0,125650.0,125650.0,12403500
2024-05-23,125650.0,125665.0,124431.0,124729.0,124729.0,9985400
2024-05-24,124731.0,125257.0,124259.0,124306.0,124306.0,9211500


In [39]:
# Função para gerar a figura gráfica
def figura_candlesticks(ohlcv_semana, ohlcv_mes, ativo):

    fig_candlesticks = make_subplots(rows=2, cols=1) # figura com 2 linhas e 1 coluna
    # configurando gráfico da semana
    fig_candlesticks.add_trace(
        go.Candlestick(
            name=str(ativo + " última semana"),
            x=ohlcv_semana.index,
            open=ohlcv_semana["Open"],
            high=ohlcv_semana["High"],
            low=ohlcv_semana["Low"],
            close=ohlcv_semana["Close"],
            increasing_line_color="green", # candles positivos = verdes
            decreasing_line_color="red", # candles negativos = vermelho
            showlegend=False,
        ),
        # definindo posição do gráfico na figura
        row=1,
        col=1,
    )

    # configurando gráfico do mês
    fig_candlesticks.add_trace(
        go.Candlestick(
            name=str(ativo + " último mês"),
            x=ohlcv_mes.index,
            open=ohlcv_mes["Open"],
            high=ohlcv_mes["High"],
            low=ohlcv_mes["Low"],
            close=ohlcv_mes["Close"],
            increasing_line_color="green",
            decreasing_line_color="red",
            showlegend=False,
        ),
        # definindo posição do gráfico na figura
        row=2,
        col=1,
    )
    # destaque visual para a última semana
    fig_candlesticks.add_vrect( # retangulo vertical
        x0=(ohlcv_semana.index[0] - timedelta(days=0.5)), # inicio do destaque
        x1=(ohlcv_semana.index[-1] + timedelta(days=0.5)), # fim do destaque
        fillcolor="gray",
        opacity=0.25,
        line_width=0,
        # definindo posição na figura, vai ficar sobre o gráfico mensal
        row=2,
        col=1,
    )
    # eliminando sábados e domingos do gráfico
    fig_candlesticks.update_xaxes(
        title_text="<b> Data",
        rangebreaks=[
            dict(bounds=["sat", "mon"]), # definindo o limite de exclusão, inicia no sábado e retoma na segunda
            # dict(bounds=[18, 10], pattern="hour"), # caso quisesse delimitar por horário
            # dict(values=["2022-06-16"]) # caso quisesse eliminar data específica, ex. feriados
        ],
    )

    fig_candlesticks.update_traces(line_width=1)

    fig_candlesticks.update_yaxes(title_text=f"<b> {ativo} (pts)")

    fig_candlesticks.update_layout(
        title_text=f"<b>Gráfico candlesticks {ativo}",
        template="simple_white",
        font=dict(family="Arial", size=20, color="black"),
    )

    fig_candlesticks.update_layout(
        margin=dict(l=20, r=20, t=70, b=20),
        paper_bgcolor="#f7f8fa",
        width=800,
        height=1200,
        xaxis_rangeslider_visible=False,
        xaxis2_rangeslider_visible=False,
    )

    return fig_candlesticks

In [40]:
grafico_candles = figura_candlesticks(ohlcv_semana, ohlcv_mes, ativo='^BVSP')

In [41]:
grafico_candles